In [ ]:
import pandas as pd
import numpy as np
import re
import sys, os

# Setup Path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from source.utils.minio_helper import read_df_from_minio

BUCKET_NAME = "mlbb-lakehouse"

# Load Data
print("📥 Loading Data...")
df_counter = read_df_from_minio(BUCKET_NAME, "bronze/counter_hero/bronze_hero_counter.parquet", file_format='parquet')
df_silver = read_df_from_minio(BUCKET_NAME, "silver/silver_draft_enriched.parquet", file_format='parquet')

# Load Helper Normalisasi (Copy dari script Anda agar konsisten)
def normalize_name_strict(text):
    if not isinstance(text, str): return ""
    clean = text.lower()
    clean = re.sub(r'[^a-z0-9]', '', clean) 
    return clean

print("✅ Data Loaded.")

📥 Loading Data...
✅ Data Loaded.


In [ ]:
print("🔬 Membangun Ulang Dictionary untuk Inspeksi...")

counter_dict = {}
sample_keys = []

if df_counter is not None:
    for t, c, s in zip(df_counter['hero_name_normalized'], df_counter['counter_name_normalized'], df_counter['score']):
        # Gunakan normalisasi strict
        k_hero = normalize_name_strict(t)
        k_counter = normalize_name_strict(c)
        counter_dict[(k_hero, k_counter)] = s
        
        # Simpan sampel untuk dilihat mata manusia
        if len(sample_keys) < 20:
            sample_keys.append(f"('{k_hero}', '{k_counter}') : {s}")

print(f"Total Rules di Dictionary: {len(counter_dict)}")
print("Contoh Key yang tersimpan:")
for k in sample_keys:
    print(f"  {k}")

🔬 Membangun Ulang Dictionary untuk Inspeksi...
Total Rules di Dictionary: 1402
Contoh Key yang tersimpan:
  ('miya', 'sora') : 7.138999938964844
  ('miya', 'belerick') : 7.111999988555908
  ('miya', 'gatotkaca') : 4.51200008392334
  ('miya', 'thamuz') : 3.494999885559082
  ('miya', 'baxia') : 3.374000072479248
  ('miya', 'terizla') : 2.4600000381469727
  ('miya', 'barats') : 1.633999943733215
  ('miya', 'fredrinn') : 0.3199999928474426
  ('miya', 'aulus') : 0.1280000060796737
  ('miya', 'luoyi') : 0.1120000034570694
  ('miya', 'granger') : 0.1110000014305114
  ('miya', 'hylos') : 0.0
  ('miya', 'lylia') : 0.0
  ('miya', 'carmilla') : 0.0
  ('balmond', 'sora') : 7.754000186920166
  ('balmond', 'cici') : 6.89300012588501
  ('balmond', 'xborg') : 6.080999851226807
  ('balmond', 'angela') : 5.787000179290772
  ('balmond', 'diggie') : 3.819999933242798
  ('balmond', 'ling') : 3.069999933242798


In [ ]:
def manual_test_score(hero_kita, list_musuh, debug=True):
    hero_norm = normalize_name_strict(hero_kita)
    musuh_norm = [normalize_name_strict(m) for m in list_musuh]
    
    total = 0
    count = 0
    
    print(f"\n🧪 Tes: {hero_kita} vs {list_musuh}")
    
    for musuh in musuh_norm:
        score = counter_dict.get((hero_norm, musuh), 0.0)
        if debug:
            found_txt = "✅ KETEMU" if (hero_norm, musuh) in counter_dict else "❌ KOSONG"
            print(f"   Lawan '{musuh}': {found_txt} | Score: {score}")
        total += score
        count += 1
        
    avg = round(total / count, 2) if count > 0 else 0
    print(f"   >> Rata-rata Score: {avg}")
    return avg

# 1. Tes Positif (Harus Tinggi)
# Phoveus vs Wanwan (Seharusnya Counter Keras)
manual_test_score("Phoveus", ["Wanwan", "Fanny", "Harith"])

# 2. Tes Negatif (Harus Rendah/Nol)
# Layla vs Tigreal (Biasanya tidak ada data khusus)
manual_test_score("Layla", ["Tigreal", "Estes"])

# 3. Tes Ejaan (Cek Normalisasi)
# Yi Sun-shin (Spasi/Strip) vs Hero Lain
manual_test_score("Yi Sun-shin", ["Lancelot"])


🧪 Tes: Phoveus vs ['Wanwan', 'Fanny', 'Harith']
   Lawan 'wanwan': ❌ KOSONG | Score: 0.0
   Lawan 'fanny': ❌ KOSONG | Score: 0.0
   Lawan 'harith': ❌ KOSONG | Score: 0.0
   >> Rata-rata Score: 0.0

🧪 Tes: Layla vs ['Tigreal', 'Estes']
   Lawan 'tigreal': ❌ KOSONG | Score: 0.0
   Lawan 'estes': ❌ KOSONG | Score: 0.0
   >> Rata-rata Score: 0.0

🧪 Tes: Yi Sun-shin vs ['Lancelot']
   Lawan 'lancelot': ❌ KOSONG | Score: 0.0
   >> Rata-rata Score: 0.0


0.0

In [ ]:
print("\n🧐 Inspeksi Top Counter Scores di Silver Layer:")

# Filter hanya yang score-nya > 0 (Phase Pick saja)
non_zero = df_silver[
    (df_silver['counter_score'] > 0) & 
    (df_silver['phase'] == 'pick')
].sort_values('counter_score', ascending=False)

if len(non_zero) > 0:
    print(f"Ditemukan {len(non_zero)} hero dengan counter score aktif.")
    
    # Tampilkan kolom relevan
    cols = ['match_id', 'team_side', 'hero_name_normalized', 'counter_score', 'role', 'lane']
    display(non_zero[cols].head(10))
    
    # Analisis Sampel Teratas
    top_row = non_zero.iloc[0]
    print(f"\nAnalisis Juara 1 (Score {top_row['counter_score']}):")
    print(f"Hero: {top_row['hero_name_normalized']} (Match {top_row['match_id']})")
    print("Kenapa skornya tinggi? Coba cek musuhnya di match itu secara manual.")
else:
    print("⚠️ Masih belum ada counter score > 0 sama sekali. Cek ulang Cell 3!")


🧐 Inspeksi Top Counter Scores di Silver Layer:
Ditemukan 1 hero dengan counter score aktif.


,match_id,team_side,hero_name_normalized,counter_score,role,lane
0,1,left,cici,0.68,fighter,exp lane



Analisis Juara 1 (Score 0.68):
Hero: cici (Match 1)
Kenapa skornya tinggi? Coba cek musuhnya di match itu secara manual.


In [ ]:
def manual_test_score_2way(hero_kita, list_musuh):
    hero_norm = normalize_name_strict(hero_kita)
    musuh_norm = [normalize_name_strict(m) for m in list_musuh]
    
    print(f"\n🧪 Tes 2 Arah: {hero_kita} (Kita) vs {list_musuh} (Musuh)")
    
    for musuh in musuh_norm:
        # Arah 1: Defensive (Apakah saya dicounter?)
        # Key: (Saya, Musuh)
        score_def = counter_dict.get((hero_norm, musuh), 0.0)
        
        # Arah 2: Offensive (Apakah saya mengcounter?)
        # Key: (Musuh, Saya)
        score_off = counter_dict.get((musuh, hero_norm), 0.0)
        
        print(f"   Vs '{musuh}':")
        print(f"     🛡️  Defensive (Key: {hero_norm},{musuh}): {score_def}")
        print(f"     ⚔️  Offensive (Key: {musuh},{hero_norm}): {score_off}")
        
        if score_off > 0:
            print(f"     ✨ HORE! {hero_kita} ternyata counter {musuh} (Offensive Score Ada!)")

# Coba lagi tes Phoveus
manual_test_score_2way("Phoveus", ["Wanwan", "Fanny"])


🧪 Tes 2 Arah: Phoveus (Kita) vs ['Wanwan', 'Fanny'] (Musuh)
   Vs 'wanwan':
     🛡️  Defensive (Key: phoveus,wanwan): 0.0
     ⚔️  Offensive (Key: wanwan,phoveus): 11.14799976348877
     ✨ HORE! Phoveus ternyata counter wanwan (Offensive Score Ada!)
   Vs 'fanny':
     🛡️  Defensive (Key: phoveus,fanny): 0.0
     ⚔️  Offensive (Key: fanny,phoveus): 0.0


In [ ]:
manual_test_score_2way("Belerick", ["Miya", "Claude"])


🧪 Tes 2 Arah: Belerick (Kita) vs ['Miya', 'Claude'] (Musuh)
   Vs 'miya':
     🛡️  Defensive (Key: belerick,miya): 0.0
     ⚔️  Offensive (Key: miya,belerick): 7.111999988555908
     ✨ HORE! Belerick ternyata counter miya (Offensive Score Ada!)
   Vs 'claude':
     🛡️  Defensive (Key: belerick,claude): 0.0
     ⚔️  Offensive (Key: claude,belerick): 0.0


In [ ]:
manual_test_score_2way("Claude", ["Lesley", "Bruno"])


🧪 Tes 2 Arah: Claude (Kita) vs ['Lesley', 'Bruno'] (Musuh)
   Vs 'lesley':
     🛡️  Defensive (Key: claude,lesley): 9.689000129699709
     ⚔️  Offensive (Key: lesley,claude): 0.0
   Vs 'bruno':
     🛡️  Defensive (Key: claude,bruno): 5.579999923706055
     ⚔️  Offensive (Key: bruno,claude): 0.0


In [ ]:
manual_test_score_2way("Lesley", ["Claude", "Estes"])


🧪 Tes 2 Arah: Lesley (Kita) vs ['Claude', 'Estes'] (Musuh)
   Vs 'claude':
     🛡️  Defensive (Key: lesley,claude): 0.0
     ⚔️  Offensive (Key: claude,lesley): 9.689000129699709
     ✨ HORE! Lesley ternyata counter claude (Offensive Score Ada!)
   Vs 'estes':
     🛡️  Defensive (Key: lesley,estes): 6.710999965667725
     ⚔️  Offensive (Key: estes,lesley): 0.0
